# 1. 處理 train data
train/test來源 ：https://github.com/wshuyi/demo-chinese-text-classification-lstm-keras/blob/master/demo.ipynb

code 來源 ：https://blog.csdn.net/Jerryzhangjy/article/details/110209984

##import

In [1]:
!pip install keras.utils
import pandas as pd


  Created wheel for keras.utils: filename=keras_utils-1.0.13-cp37-none-any.whl size=2657 sha256=060291b29c4fb52de28eca967ae39fd07a395c9d6929a8e337fad3376aa66ce4
  Stored in directory: /root/.cache/pip/wheels/46/25/27/7707005c1cb27e1ffc8277b004ac295e34767b02b44d73d6be
Successfully built keras.utils


In [2]:
! pip3 install transformers
! pip3 install keras
! pip3 install tensorflow

     |████████████████████████████████| 2.5MB 15.2MB/s 
     |████████████████████████████████| 901kB 38.6MB/s 
     |████████████████████████████████| 3.3MB 30.8MB/s 


In [3]:
import torch
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


##讀檔

In [4]:
!git clone https://github.com/wshuyi/demo-chinese-text-classification-lstm-keras.git

Cloning into 'demo-chinese-text-classification-lstm-keras'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.


In [5]:
from pathlib import Path
mypath = Path("demo-chinese-text-classification-lstm-keras")

In [6]:
#-------------Modify the train_path----------------

train_path = mypath/'dianping.csv'

#-----------------------------------
df = pd.read_csv(train_path)
df.head(3)

,comment,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,0
1,菜品丰富质量好，服务也不错！很喜欢！,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,0


# Bert

In [7]:
import numpy as np
import pandas as pd
T_sentences = df.comment.values # T means Total
T_labels = np.array(df.sentiment.values,dtype=np.int32)

In [8]:
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")


Loading BERT tokenizer...


## training data

In [9]:
LS = len(T_sentences)
training_samples = (int)(LS * 0.9)
sentences = T_sentences #[:training_samples]
labels = T_labels #[:training_samples]

#test_sentences = T_sentences[ training_samples : LS ]
#test_labels = T_labels[training_samples : LS]


In [10]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)
# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Original:  口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说菜都是提前做好的。几乎都不热。菜品：酸汤肥牛，干辣干辣的，还有一股泡椒味。着实受不了。。。环境：室内整体装修确实不错，但是大厅人多，太乱。服务：一般吧。说不上好，但是也不差。价格：一般，大众价格，都能接受。人：太多了，排队很厉害。以后不排队也许还会来（比如早去路过），排队就不值了。票据：六日没票！告我周一到周五可能有票~~~ 相当不正规，在这一点同等价位远不如外婆家。
Token IDs: [101, 1366, 1456, 8038, 679, 4761, 6887, 3221, 2769, 1366, 7770, 749, 8024, 6820, 3221, 6821, 2157, 4696, 679, 2582, 720, 3416, 511, 2769, 2697, 6230, 1366, 1456, 4802, 2141, 2523, 671, 5663, 2523, 671, 5663, 511, 677, 5831, 4685, 2496, 2571, 8024, 2769, 3140, 6432, 5831, 6963, 3221, 2990, 1184, 976, 1962, 4638, 511, 1126, 725, 6963, 679, 4178, 511, 5831, 1501, 8038, 7000, 3739, 5503, 4281, 8024, 2397, 6793, 2397, 6793, 4638, 8024, 6820, 3300, 671, 5500, 3796, 3492, 1456, 511, 4708, 2141, 1358, 679, 749, 511, 511, 511, 4384, 1862, 8038, 2147, 1079, 3146, 860, 6163, 934, 4802, 2141, 679, 7231, 8024, 852, 3221, 1920, 1324, 782, 1914, 8024, 1922, 744, 511, 3302, 1218, 8038, 671, 5663, 1416, 511, 6432, 679, 677, 1962, 8024, 852, 3221, 738, 

In [11]:

from keras.preprocessing.sequence import pad_sequences
# Set the maximum sequence length.
MAX_LEN = 200
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# Pad our input tokens with value 0.
# 注意 ："post" indicates that we want to pad and truncate at the end of the sequence,
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.')


Padding/truncating all sentences to 200 values...

Padding token: "[PAD]", ID: 0
\Done.


In [12]:
# Create attention masks
attention_masks = []
# For each sentence...
for sent in input_ids:
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [13]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [14]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [15]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# For fine-tuning BERT on a specific task, the authors recommend a batch size of # 16 or 32.
batch_size = 16
# Create the DataLoader for our training set.
print(train_inputs.size(),train_masks.size(),train_labels.size())
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

torch.Size([1800, 200]) torch.Size([1800, 200]) torch.Size([1800])


In [16]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "hfl/chinese-bert-wwm-ext",
    num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
#     "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
 

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [17]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (21128, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [18]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
epochs = 4 # Number of training epochs (authors recommend between 2 and 4)
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
print(total_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

452


In [19]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [20]:
import random
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0

    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train() #just changes the *mode*, it doesn't *perform* the training.
    #----------------------- For each batch of training data -----------------------
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)           
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].long().to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].long().to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch) # Add batch to GPU
        b_input_ids, b_input_mask, b_labels = batch # Unpack the inputs from our dataloader
        
        # Tell the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():        
            outputs = model(b_input_ids, token_type_ids=None,attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:41.
  Batch    80  of    113.    Elapsed: 0:01:22.

  Average training loss: 0.36
  Training epcoh took: 0:01:56

Running Validation...
  Accuracy: 0.88
  Validation took: 0:00:05

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:41.
  Batch    80  of    113.    Elapsed: 0:01:22.

  Average training loss: 0.24
  Training epcoh took: 0:01:56

Running Validation...
  Accuracy: 0.90
  Validation took: 0:00:05

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:41.
  Batch    80  of    113.    Elapsed: 0:01:23.

  Average training loss: 0.13
  Training epcoh took: 0:01:56

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:05

======== Epoch 4 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:00:41.
  Batch    80  of    113.    Elapsed: 0:01:22.

  Average training loss: 0.07
  Training epcoh took: 0:01:55

Ru

In [21]:
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

## Tesing dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
path =  "/content/drive/Shareddrives/AI_project/"
df_test = pd.read_csv(path + 'data/1_column_title/simple_label_2kind.csv')
test_sentences = df_test['title']
test_labels = df_test['label']

In [ ]:
import pandas as pd
# Load the dataset into a pandas dataframe.
'''
# Create sentence and label lists
sentences = df.sentence.values
labels = np.array(df.label.values,dtype=np.int32)
'''

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in test_sentences:
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True, )  
    input_ids.append(encoded_sent)

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(test_labels)

# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')
print('Positive samples: %d of %d (%.2f%%)' % (test_labels.sum(), len(test_labels), (test_labels.sum() / len(test_labels) * 100.0)))

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []
# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')
# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)
  # Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]
# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
print('MCC: %.3f' % mcc)

#-------------------------------------------------------------------------

count = 0
for i in range(len(flat_true_labels)):
  if int(flat_predictions[i]) ==  int(flat_true_labels[i]):
      count +=1
  else:
    None
    #print("title : ",test_sentences[i], flat_true_labels[i]," wrong label ",flat_predictions[i])
print(count, "正确率： " ,count/len(flat_true_labels))

#利用原始Model_output

In [ ]:
!pip install snownlp
from snownlp import SnowNLP

     |████████████████████████████████| 37.6MB 89kB/s 
  Created wheel for snownlp: filename=snownlp-0.12.3-cp37-none-any.whl size=37760967 sha256=09fb43c6926693b1e37cb10637dea2f4771a33a8c06f639ae92f58521b073e97
  Stored in directory: /root/.cache/pip/wheels/f3/81/25/7c197493bd7daf177016f1a951c5c3a53b1c7e9339fd11ec8f
Successfully built snownlp


In [ ]:
media = "udn"
sample_cnt = 200
'''
udn, storm
sector.size :
  chinatimes     9129
  cna            4060
  cts            1052
  ebc            1414
  ettoday        8469
  ltn            8518
  media             1
  setn           5452
  storm          1281 : ['公共政策', '國內'] 
  udn           11279 : 社會
'''

"\nudn, storm\nsector.size :\n  chinatimes     9129\n  cna            4060\n  cts            1052\n  ebc            1414\n  ettoday        8469\n  ltn            8518\n  media             1\n  setn           5452\n  storm          1281 : ['公共政策', '國內'] \n  udn           11279 : 社會\n"

In [ ]:
path =  "/content/drive/Shareddrives/AI_project/"
df = pd.read_csv(path + 'data/a.csv')
#df_3col = df[['media', 'category', 'title']]
df_3col = df[['media', 'title','point']]
#sector = df_3col.groupby('media')
#sector.size()
#m = sector.get_group(media) #m=media
#mc = m.groupby('category') #mc=media category
#mc_P = mc.get_group("['公共政策', '國內']")['title'] # mc_P's P means politic
mc_P = df_3col['title']

data_array = []
for line in mc_P : 
  simple = SnowNLP(line).han #轉簡體
  data_array.append(simple)
data = pd.DataFrame(data_array, columns = ['title'])
#--------------------------------------------------------------

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in data_array:
    encoded_sent = tokenizer.encode(sent,add_special_tokens = True,)    
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(test_labels)
# Set the batch size.  
batch_size = 32  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

#--------------------------------------------------------------
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions = []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  '''label_ids = b_labels.to('cpu').numpy()'''
  
  # Store predictions and true labels
  predictions.append(logits)
  '''true_labels.append(label_ids)'''
print('DONE.')
#--------------------------------------------------------------
matthews_set = []
print('Calculating Matthews Corr. Coef. for each batch...')
# For each input batch...
for i in range(len(predictions)):
  
  #Pick the label with the highest value and turn this in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(pred_labels_i, pred_labels_i)                
  matthews_set.append(matthews)

flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()


In [ ]:
pos_idx = []
neg_idx  = [] 
pos_cnt = 0
neg_cnt = 0

for i in range(len(flat_predictions)):
  predict = flat_predictions[i]
  if (predict==0) : 
    neg_cnt+=1
    neg_idx.append(i)  
  else:
    pos_cnt+=1
    pos_idx .append(i)
#----------------------------------------------------------------
path =  "/content/drive/Shareddrives/AI_project/"
filepath = path+'result/a'

path = filepath+'.txt'
f = open(path, 'w')
f.write(str(pos_cnt)+" ")
f.close()
f = open(path, 'a')
f.write(str(neg_cnt))
f.close()
#----------------------------------------------------------------
pos = []
neg = [] 
for idx in pos_idx : pos.append(data_array[idx])
for idx in neg_idx : neg.append(data_array[idx])

p = pd.DataFrame(pos)
p.to_csv(filepath+"pos.csv")


neg = pd.DataFrame(neg)
neg.to_csv(filepath+"neg.csv")


In [ ]:
pos_u = pos.unique

In [ ]:
print("positive cnt : ",pos_cnt," negative cnt : ", neg_cnt)
print()
percentage = (pos_cnt+0.0)/(pos_cnt+neg_cnt)
print( "percentage of positive title","{:.2f}".format(percentage),"%" )

nega =  neg[0][10:20]

positive cnt :  69  negative cnt :  109

percentage of positive title 0.39 %


In [ ]:
neg_unique = []
pos_unique = []
for idx in pos_idx:
  pos_unique.append(str(df_3col.loc[idx]['media'])+" "+str(df_3col.loc[idx]['title'])+" "+str(df_3col.loc[idx]['point']))

for idx in neg_idx:
  neg_unique.append(str(df_3col.loc[idx]['media'])+" "+str(df_3col.loc[idx]['title'])+" "+str(df_3col.loc[idx]['point']))



In [ ]:
for n in pos_unique:
  print(n)

udn 美赠250万剂疫苗 外交部：展现对台坚定支持与高度重视 1.0
cna 美赠250万剂疫苗  吴钊燮：台湾人永远铭记在心 0.9139534235000609
chinatimes 美赠250万剂疫苗 萧美琴：加倍的爱启程运往台湾 0.8897879123687744
cna 美赠250万剂疫苗将抵台 吴钊燮：台美关系坚若磐石 0.8743146657943726
ltn 美国赠台250万剂  张丽善：及时雨解决疫苗荒 0.8694922924041748
ltn 美250万剂疫苗今抵台  郑文灿：台美真朋友 0.8530252575874329
ltn 白宫：本月完成分配8000万剂疫苗 即刻出货 0.8435049653053284
ltn 美赠250万剂莫德纳疫苗 黄敏惠：真的非常感动 0.8361530900001526
chinatimes 白宫：未来几天分配完8000万剂疫苗 即刻出货 0.7759473323822021
cna 白宫：未来几天分配完8000万剂疫苗 即刻出货 0.7759473323822021
udn 白宫：未来几天分配完8000万剂疫苗 即刻出货 0.7759473323822021
cts 美赠250万剂疫苗 绿县市首长发文致谢 0.8288391828536987
cts 美国加码赠台250万剂疫苗 民进党：非常温暖 0.8181829452514648
udn 美赠250万剂疫苗 王定宇：化解中国打压台湾压力 0.8167411088943481
cna 美国赠250万剂疫苗 总统：印证台美友好坚若磐石 0.8158669471740723
chinatimes 美国加码捐赠250万剂疫苗 行政院：患难之际相助心存感激 0.8149165511131287
cna 美捐赠疫苗250万剂 江启臣：印证台美情谊 0.8140252828598022
ltn 美国赠我250万剂疫苗 苏贞昌：非常感动  0.8126640915870667
cna 美国赠250万剂疫苗 苏贞昌：真的非常感动 0.8124833106994629
udn 美国250万剂疫苗将抵台 陈其迈：台美互信互助情谊深厚 0.8016114830970764
cna 美捐赠疫苗追加至250万剂 AIT：台湾是可信赖的朋友 0.79008615016937